
# ⚡ Using **GPT-5-mini** — Colab 실습 노트북
이 노트북은 OpenAI **Responses API**에서 **GPT-5-mini**를 중심으로 실습합니다.  
핵심: **reasoning.effort**, **text.verbosity**, **max_output_tokens** (GPT-5 계열: `temperature/top_p/logprobs` 미지원).

> 접근 권한/리전 이슈가 있을 수 있으므로, 불가 시 자동으로 `gpt-4o-mini`로 **폴백**합니다.



## 1) 환경 셋업
- 최신 OpenAI Python SDK 설치


In [ ]:

!pip -qU install openai>=1.55.0



## 2) API 키 설정
Colab **Settings → Variables**에 `OPENAI_API_KEY` 저장 권장. 없으면 아래 셀에서 입력.


In [ ]:

import os
from getpass import getpass

OPENAI_API_KEY = None
try:
    from google.colab import userdata  # type: ignore
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except Exception:
    pass

if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OPENAI_API_KEY: ")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print("✅ API key set.")



## 3) 클라이언트 초기화 & 모델 선택 (폴백 포함)
- 기본 타깃: `gpt-5-mini`  
- 실패 시: `gpt-4o-mini`


In [ ]:

from openai import OpenAI
client = OpenAI()

PRIMARY_MODEL = "gpt-5-mini"
FALLBACK_MODEL = "gpt-4o-mini"

def pick_model():
    try:
        _ = client.responses.create(
            model=PRIMARY_MODEL,
            input="ping",
            reasoning={"effort": "minimal"},
            max_output_tokens=5,
            text={"verbosity": "low"},
        )
        return PRIMARY_MODEL
    except Exception:
        print("[warn] gpt-5-mini not available → falling back to gpt-4o-mini")
        return FALLBACK_MODEL

MODEL = pick_model()
print("Using model:", MODEL)



## 4) Quickstart — 빠른 응답(저지연)
- `reasoning.effort`: `"low"` 또는 `"minimal"`  
- `text.verbosity`: `"low"`  
- `max_output_tokens`: 응답 길이 제어


In [ ]:

resp = client.responses.create(
    model=MODEL,
    input="Write a haiku about clean, elegant Python code.",
    reasoning={"effort": "low"},
    text={"verbosity": "low"},
    max_output_tokens=80,
)
print(resp.output_text)
try:
    print("\n[usage]", resp.usage)
except Exception:
    pass



## 5) 고난도 태스크 — 높은 추론 노력
복잡한 버그 분석/계획 수립에는 `reasoning.effort="high"`를 사용합니다.


In [ ]:

prompt = "Find the bug and fix it. Code:\n\nfunction add(a, b) { if (!a) return; return a + b; }"
resp = client.responses.create(
    model=MODEL,
    input=prompt,
    reasoning={"effort": "high"},
    text={"verbosity": "medium"},
    max_output_tokens=400,
)
print(resp.output_text)



## 6) Minimal reasoning — 극저지연
추론 토큰을 최소화해 즉답에 가깝게 응답.


In [ ]:

resp = client.responses.create(
    model=MODEL,
    input="How many letters r are in 'blueberry'? Return only the integer.",
    reasoning={"effort": "minimal"},
    text={"verbosity": "low"},
    max_output_tokens=10,
)
print(resp.output_text)



## 7) Verbosity 비교 실험
같은 질문으로 `low/medium/high` 비교


In [ ]:

question = "Explain what a binary search tree is, and show a short Python example."
for v in ["low", "medium", "high"]:
    print(f"\n=== verbosity={v} ===")
    resp = client.responses.create(
        model=MODEL,
        input=question,
        reasoning={"effort": "low"},
        text={"verbosity": v},
        max_output_tokens=400,
    )
    print(resp.output_text[:1200])



## 8) 헬퍼 함수 — `ask5mini()`
반복 실험을 간편화합니다.


In [ ]:

def ask5mini(prompt: str,
             reasoning_effort: str = "medium",
             verbosity: str = "medium",
             max_tokens: int = 400,
             system_prompt: str | None = None):
    payload = []
    if system_prompt:
        payload.append({"role": "system", "content": system_prompt})
    payload.append({"role": "user", "content": prompt})
    r = client.responses.create(
        model=MODEL,
        input=payload,
        reasoning={"effort": reasoning_effort},
        text={"verbosity": verbosity},
        max_output_tokens=max_tokens,
    )
    return r

demo = ask5mini("List 3 benefits of Responses API succinctly.", "low", "low", 120)
print(demo.output_text)



## 9) 구조화(JSON) 응답
`response_format={"type":"json_object"}`로 파싱 가능한 결과 받기


In [ ]:

import json

prompt = "Summarize this in <=2 sentences and extract 3 keywords: GPT-5-mini balances speed, cost, and capability."
resp = client.responses.create(
    model=MODEL,
    input=prompt,
    response_format={"type": "json_object"},
    text={"verbosity": "low"},
    max_output_tokens=120,
)
print("Raw:", resp.output_text)
try:
    print("Parsed:", json.loads(resp.output_text))
except Exception as e:
    print("JSON parse error:", e)



## 10) (선택) 스트리밍 출력
토큰 단위로 점진적 출력


In [ ]:

from contextlib import contextmanager

@contextmanager
def response_stream(**kwargs):
    s = client.responses.stream(**kwargs)
    try:
        yield s
    finally:
        s.close()

with response_stream(
    model=MODEL,
    input="Write a 5-line poem about on-device acceleration.",
    reasoning={"effort": "low"},
    text={"verbosity": "low"},
    max_output_tokens=120,
) as stream:
    for event in stream:
        if event.type == "response.output_text.delta":
            print(event.delta, end="")
        elif event.type == "response.completed":
            print("\n\n[completed]")



## 11) (참고) Custom tools / Allowed tools — 요청 샘플
수업에서는 개념과 보안(서버 측 검증)을 강조하고, 실제 실행은 서버 샌드박스에서 다루세요.


In [ ]:

example_payload_custom_tool = {
    "model": "gpt-5-mini",
    "input": "Use the code_exec tool to calculate area of a circle with r=3.",
    "tools": [
        {"type": "custom", "name": "code_exec", "description": "Executes arbitrary python code"}
    ]
}
example_payload_allowed_tools = {
    "model": "gpt-5-mini",
    "input": "Search docs for 'Responses API' and summarize.",
    "tools": [
        {"type": "function", "name": "search_docs", "description": "Searches internal docs by keyword"},
        {"type": "function", "name": "get_weather", "description": "Gets current weather"}
    ],
    "tool_choice": {
        "type": "allowed_tools",
        "mode": "auto",
        "tools": [
            {"type": "function", "name": "search_docs"}
        ]
    }
}
print("Custom tool sample:", example_payload_custom_tool)
print("\nAllowed tools sample:", example_payload_allowed_tools)



---
## 🧪 실습 과제
1) **지연/품질 트레이드오프**: 동일 질문에 대해  
   - `reasoning.effort`: minimal/low/medium/high  
   - `text.verbosity`: low/medium/high  
   3개 이상 조합 실험 후, **속도/길이/품질 차이**를 5줄 요약.

2) **코딩 디버깅**: 10~20줄 오류 코드로 `reasoning.effort="high"` 테스트.  
   결과 코드를 별도 환경에서 실행해 동작 여부 확인(보고만).

3) **JSON 스키마 설계**: 요약/키워드/난이도(1~5)/추가자료 링크 포함 JSON으로 두 문단 비교.



---
_Last updated: 2025-11-06 14:05:57_  
© Ajou Univ. **모바일 컴퓨팅 특론** — Using GPT-5-mini Lab
